<a href="https://colab.research.google.com/github/HaloSigner/MOFA-_practice/blob/main/mofa%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import scanpy as sc
import muon as mu
import mofax as mofa
import numpy as np
import pandas as pd

In [ ]:
import os

In [ ]:
obs = pd.read_csv('CLL_metadata.tsv', sep='\t', index_col='sample')
obs

In [ ]:
mods = {i:sc.AnnData(pd.read_csv(f'{i}.csv', index_col=0).T) for i in ('Drugs', 'Methylation', 'mRNA', 'Mutations')}
mods

In [ ]:
for i in ['Drugs', 'Methylation', 'mRNA', 'Mutations']:
    i = pd.read_csv(f'{i}.csv')
    display(i)

In [ ]:
mdata=mu.MuData(mods)
mu.set_options(pull_on_update=False)

mdata.obs = mdata.obs.join(obs)

In [ ]:
mdata

# Integrate data #

In case some samples are missing from some of the modalities, as they are in this CLL dataset, muon cna be instructed to handle it. One of two modes has to be chosen for that:

1. use_obs='union', which will populate values for missing samples in certain modalities with NAs, or
2. use_obs='intersection', which will use a subset of samples that are present in all modalities


In [ ]:
mu.tl.mofa(mdata, use_obs='union',
           n_factors=10, convergence_mode='medium',
           outfile='models/CLL_union.hdf5')

In [ ]:
mdata.obs['IGHV_status'] = mdata.obs['IGHV'].astype(str).astype('category').cat.rename_categories({"1.0": "mutated", "0.0": "unmutated"})
mu.pl.mofa(mdata, color='IGHV_status')

In [ ]:
mdata.obs['trisomy12_status'] = mdata.obs['trisomy12'].astype(str).astype("category").cat.rename_categories({"1.0": "mutated", "0.0": "unmutated"})
mu.pl.mofa(mdata, color="trisomy12_status")

In [ ]:
model = mofa.mofa_model('models/CLL_union.hdf5')
model

In [ ]:
mofa.plot_r2(model, x='View', vmax=10)

In [ ]:
mofa.plot_weights_correlation(model)

In [ ]:
ax = mofa.plot_weights(model, n_features=10,
                       y_repel_coef=0.04, x_rank_offset=-150)

In [ ]:
mofa.plot_weights_heatmap(model, n_features=5, factors=range(0,10), w_abs=True, cmap='viridis', cluster_factors=False, view=['Mutations'])

In [ ]:
mofa.plot_weights_scatter(model, x="Factor1", y="Factor2",view=["Drugs"],
                         hist=True, n_features=20)

In [ ]:
mofa.plot_weights(model, views=['mRNA'], factors=0, zero_line=True, ncols=2, label_size=10)

In [ ]:
mofa.plot_weights(model, views=[ 'Mutations'], factors=0, zero_line=True, ncols=2, label_size=10)

In [ ]:
mofa.plot_weights(model, views=['Drugs'], factors=0, label_size=10)

In [ ]:
mofa.plot_weights_heatmap(model, n_features=25,  factors=0, w_abs=True, cmap='viridis', cluster_factors  = False, view=["mRNA"])